### Illumina read processing and taxonomic classification of query sequences ##

We are using our custom pipeline [metaBEAT](https://github.com/HullUni-bioinformatics/metaBEAT) to process the Illumina data and taxonomically identify query sequences. 

For full reproducibility metaBEAT was run inside a docker container - [here](https://hub.docker.com/r/chrishah/metabeat/).

After initial read quality trimming, -merging and clustering, query sequences are blasted against a custom reference database composed of COI sequences of _Gammarus_ sp. as well as the positive controls _Harmonia axyridis_ and _Triops cancriformis_ (all downloaded from Genbank as described [here]()). Taxonomic assignment was perfored using a lowest commong ancestor (LCA) approach based on the BLAST results, as described in the paper.

The file `Querymap.txt` contains the sampleIDs and the location of the Illumina read files, plus the barcodes and instructions to clip off the first 30 bases of both the forward and reverse reads, in order to remove any primers.

The file `REFlist.txt` points towards the reference sequences.

In [3]:
!head QUERYmap.txt

INV001D	fastq	../../raw_data/Rosie/INV001D_S1_L001_R1_001.fastq.gz	../../raw_data/Rosie/INV001D_S1_L001_R2_001.fastq.gz	TCGCCTTA	TAGATCGC	30	30
INV002D	fastq	../../raw_data/Rosie/INV002D_S2_L001_R1_001.fastq.gz	../../raw_data/Rosie/INV002D_S2_L001_R2_001.fastq.gz	TCGCCTTA	CTCTCTAT	30	30
INV003D	fastq	../../raw_data/Rosie/INV003D_S3_L001_R1_001.fastq.gz	../../raw_data/Rosie/INV003D_S3_L001_R2_001.fastq.gz	TCGCCTTA	TATCCTCT	30	30
INV004D	fastq	../../raw_data/Rosie/INV004D_S4_L001_R1_001.fastq.gz	../../raw_data/Rosie/INV004D_S4_L001_R2_001.fastq.gz	TCGCCTTA	AGAGTAGA	30	30
INV005	fastq	../../raw_data/Rosie/INV005_S5_L001_R1_001.fastq.gz	../../raw_data/Rosie/INV005_S5_L001_R2_001.fastq.gz	TCGCCTTA	GTAAGGAG	30	30
INV006	fastq	../../raw_data/Rosie/INV006_S6_L001_R1_001.fastq.gz	../../raw_data/Rosie/INV006_S6_L001_R2_001.fastq.gz	TCGCCTTA	ACTGCATA	30	30
INV007D	fastq	../../raw_data/Rosie/INV007D_S7_L001_R1_001.fastq.gz	../../raw_data/Rosie/INV007D_S7_L001_R2_001.fastq.gz	TCGCCTTA	AAGGAGTA	30	3

Prepare list of reference files.

In [5]:
%%bash

for gb in $(ls -1 ../../1-download_references/*.gb | grep "refseqs" -v)
do
    echo -e "$gb\tgb"
done > REFlist.txt

In [6]:
!cat REFlist.txt

../../1-download_references/Gammarus_COI_Weiss_et_al_2013.gb	gb
../../1-download_references/positive_controls.gb	gb


Run the metaBEAT pipeline.

In [7]:
!metaBEAT_global.py --version

0.97.7-global


In [8]:
!metaBEAT_global.py \
-Q QUERYmap.txt \
-R REFlist.txt \
--trim_minlength 100 \
--trim_qual 30 \
--merge --product_length 350 \
--forward_only \
--cluster --clust_match 0.97 --clust_cov 3 \
--length_filter 313 --length_deviation 0.05 \
--blast --min_ident 0.95 \
-m COI -n 3 -v -@ c.hahn@hull.ac.uk \
-o COI_160307_clip_trim-30_merge_forw-only_c0.97m3_blast_min0.85_GLOBAL > metaBEAT.log

A summary of the read counts throughout the read processing stages can be found in the file:

`COI_160307_clip_trim-30_merge_forw-only_c0.97m3_blast_min0.85_GLOBAL_read_stats.csv`.

The final OTU table can be found in the file: 

`GLOBAL/BLAST_0.95/COI_160307_clip_trim-30_merge_forw-only_c0.97m3_blast_min0.85_GLOBAL-by-taxonomy-readcounts.blast.tsv`.


Extract from read stats file the proportions of reads have been removed by the minimum cluster size filter.

In [71]:
import numpy as np

rp = []
cp = []

fh = open('COI_160307_clip_trim-30_merge_forw-only_c0.97m3_blast_min0.85_GLOBAL_read_stats.csv','r')
fh.next()

for l in fh:
    r_prefilter = int(l.strip().split(",")[5])/2
    c_prefilter = int(l.strip().split(",")[7])
    r_postfilter = int(l.strip().split(",")[-1])
    c_postfilter = int(l.strip().split(",")[-2])
    
#    print r_prefilter,r_postfilter
    if not r_postfilter == 0:
#        print r_prefilter,r_postfilter,100-float(r_postfilter)/r_prefilter*100
#        print c_prefilter,c_postfilter,100-float(c_postfilter)/c_prefilter*100
        rp.append(100-float(r_postfilter)/r_prefilter*100)
        cp.append(100-float(c_postfilter)/c_prefilter*100)
        
print "average proportion of reads removed by filter: %.3f %%" %np.average(rp)
print "average proportion of clusters removed by filter: %.3f %%" %np.average(cp)


average proportion of reads removed by filter: 12.495 %
average proportion of clusters removed by filter: 70.202 %


In [72]:
import metaBEAT_global_misc_functions as mb

Identify sites which contained reads assigned to _G. fossarum_ before filtering.

In [73]:
mb.find_target(BIOM=mb.load_BIOM('GLOBAL/BLAST_0.95/COI_160307_clip_trim-30_merge_forw-only_c0.97m3_blast_min0.85_GLOBAL-by-taxonomy-readcounts.blast.biom'), 
               target='Gammarus_fossarum')


Specified BIOM input format 'json' - ok!
BLANK-1.blast	(16.9811 %)
INV004D.blast	(0.3456 %)
INV005.blast	(99.8875 %)
INV010.blast	(98.8542 %)
INV019.blast	(0.2357 %)
INV021.blast	(0.1204 %)
INV027.blast	(93.5046 %)
INV028.blast	(89.6334 %)
INV029.blast	(100.0000 %)
INV030.blast	(99.5250 %)
INV031D.blast	(19.8005 %)
INV033.blast	(1.6838 %)
INV034D.blast	(11.3552 %)
INV035.blast	(100.0000 %)
INV036.blast	(99.9187 %)
INV037.blast	(99.6670 %)
INV038.blast	(100.0000 %)
INV039.blast	(100.0000 %)
INV040.blast	(98.7659 %)
INV041.blast	(99.8786 %)
INV042.blast	(76.3237 %)
INV043.blast	(0.1853 %)
INV049.blast	(5.0641 %)
INV053.blast	(95.4602 %)
INV055.blast	(99.6733 %)
INV056.blast	(94.5860 %)
INV057.blast	(99.6347 %)
INV058.blast	(0.2272 %)
INV059.blast	(99.2910 %)
INV060.blast	(0.8555 %)
INV062.blast	(97.4181 %)
INV063.blast	(3.0410 %)
SOI005.blast	(12.0603 %)
SOI024.blast	(0.3726 %)
SOI028.blast	(0.3363 %)
SOI029.blast	(1.6717 %)
SOI030.blast	(0.0120 %)
SOI031.blast	(0.0069 %)
SOI032.blast	(

Filter raw OTU table - in a given sample remove OTUs that were not supported by at least 1% of the reads.

In [81]:
#load raw OTU table
to_filter = mb.load_BIOM(table='GLOBAL/BLAST_0.95/COI_160307_clip_trim-30_merge_forw-only_c0.97m3_blast_min0.85_GLOBAL-OTU-taxonomy.blast.biom')

#filter at 1%
filtered = mb.filter_BIOM_by_per_sample_read_prop(BIOM=to_filter, min_prop=0.01)

#write to file
mb.write_BIOM(filtered, target_prefix='filtered' )

#collapse OTUs by taxonomy
filtered_collapsed = mb.collapse_biom_by_taxonomy(in_table=filtered)

#write to file
mb.write_BIOM(filtered_collapsed, target_prefix='filtered-collapsed')


Specified BIOM input format 'json' - ok!

Filtering at level: 1.0 %

Removing 5600 OTUs for lack of support

Writing 'filtered.biom'
Writing 'filtered.tsv'
Writing 'filtered-collapsed.biom'
Writing 'filtered-collapsed.tsv'


Identify samples containing sequences assigned to _G. fossarum_.

In [77]:
mb.find_target(filtered_collapsed, target='Gammarus_fossarum')

BLANK-1.blast	(16.9811 %)
INV005.blast	(100.0000 %)
INV010.blast	(100.0000 %)
INV027.blast	(95.2122 %)
INV028.blast	(90.8974 %)
INV029.blast	(100.0000 %)
INV030.blast	(100.0000 %)
INV031D.blast	(20.4174 %)
INV033.blast	(1.7705 %)
INV034D.blast	(11.9880 %)
INV035.blast	(100.0000 %)
INV036.blast	(100.0000 %)
INV037.blast	(100.0000 %)
INV038.blast	(100.0000 %)
INV039.blast	(100.0000 %)
INV040.blast	(100.0000 %)
INV041.blast	(100.0000 %)
INV042.blast	(77.0550 %)
INV049.blast	(5.4598 %)
INV053.blast	(98.1144 %)
INV055.blast	(100.0000 %)
INV056.blast	(94.5860 %)
INV057.blast	(100.0000 %)
INV059.blast	(100.0000 %)
INV062.blast	(98.7500 %)
INV063.blast	(3.2258 %)
SOI005.blast	(12.7321 %)
SOI029.blast	(1.8174 %)
SOI035.blast	(8.1596 %)
SOI037.blast	(1.7464 %)
SOI038.blast	(3.4930 %)
SOI039.blast	(6.1993 %)
SOI040.blast	(1.8638 %)
SOI057.blast	(25.7233 %)
SOI062.blast	(3.6036 %)
WAT029.blast	(7.9436 %)
WAT035.blast	(11.7241 %)
WAT036.blast	(7.8764 %)
WAT037.blast	(24.8268 %)
WAT038.blast	(4.8209

Identify samples containing sequences assigned to _G. fossarum_.

In [78]:
mb.find_target(filtered_collapsed, target='Gammarus_pulex')

BLANK-1.blast	(62.8931 %)
INV011.blast	(81.1340 %)
INV013.blast	(20.7633 %)
INV015.blast	(16.6021 %)
INV016.blast	(56.4469 %)
INV017.blast	(3.2396 %)
INV018.blast	(8.0049 %)
INV019.blast	(18.4542 %)
INV021.blast	(68.8352 %)
INV023.blast	(100.0000 %)
INV025.blast	(50.0000 %)
INV026.blast	(39.3834 %)
INV042.blast	(5.1437 %)
INV043.blast	(54.0541 %)
INV044.blast	(16.1581 %)
INV046.blast	(4.1423 %)
INV048.blast	(12.3082 %)
INV049.blast	(5.8096 %)
INV050.blast	(1.8329 %)
INV051.blast	(91.4938 %)
INV052.blast	(70.9160 %)
INV054.blast	(65.6038 %)
INV056.blast	(4.1401 %)
INV058.blast	(1.4144 %)
INV060.blast	(76.6568 %)
INV061.blast	(15.2616 %)
INV063.blast	(20.0000 %)
INV064.blast	(4.0670 %)
